# Initial attempts: Conversion of DCL Calcium Imaging data into NWB:N
---
### Current status:
We´re still quite at the beginning - so no widget can be found here yet :-) <br>
ELN roll-out will start in the upcoming weeks, and we are thus starting with the conversion of files with manual input of the data
---

### The file will also contain data about:
* Calcium imaging
* scored behaviors
* Heart rate recording
* Thermal imaging

In [3]:
# Import all dependencies from the NWB environment
%matplotlib widget

import datetime
import pytz
import string
import random

import pandas as pd
import numpy as np
import h5py
import math
import os

from skimage import io
from skimage.draw import polygon

import matplotlib.pyplot as plt

from nwbwidgets import nwb2widget

from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.file import Subject
from pynwb.device import Device
from pynwb.behavior import SpatialSeries, Position, BehavioralEpochs
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, Fluorescence

## Load the data that will be converted (and merged) into a NWB file:

In [5]:
file_dir = '/home/ds/NWB/ExampleDataset/'

In [6]:
# Tracking, scored behavioral events, ROI contours, fluorescence traces
d_dfs = pd.read_excel(file_dir + '175_F7-49_201030_OF_AllData.xls', sheet_name=None)
# Raw calcium imaging movie
f = h5py.File(file_dir + '175_F7-49_201030_OF_PP-1_PF-1_MC-1.h5', 'r')
#img_stack = io.imread('175_F7-49_201030_OF_PP.tiff')

# For dummy thermal trace:
df_states = pd.read_csv(file_dir + 'States_ceiling_reduced.csv', index_col=0)


### Create binary segmentation masks for all ROIs individually

In [16]:
l_ROI_IDs = [elem[:-2] for elem in d_dfs['CAI - ROIS'].columns[::2]]
l_ROI_masks = []

for ROI_ID in l_ROI_IDs:
    
    x = d_dfs['CAI - ROIS']['{}_X'.format(ROI_ID)].values
    last_idx = np.where(~np.isnan(x))[0][-1] + 1
    x = x[:last_idx]
    y = d_dfs['CAI - ROIS']['{}_Y'.format(ROI_ID)].values[:last_idx]
    xx, yy = polygon(x,y)
    ROI_mask = np.zeros((348, 385))
    ROI_mask[xx, yy] = 1
    l_ROI_masks.append((ROI_ID, ROI_mask))

### Extract tracked position

In [17]:
x = d_dfs['Tracking']['CenterG_X'].values
y = d_dfs['Tracking']['CenterG_Y'].values

times = d_dfs['Tracking']['Times'].values


position_data = np.array((x,y)).T
position_times = d_dfs['Tracking']['Times'].values

### Extract interval of behavioral events

In [18]:
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return idx-1
    else:
        return idx
    
    
def take2nd(elem):
    return elem[1]

In [19]:
l_behaviors = [elem[:elem.index('_')] for elem in d_dfs['Behaviour'].columns[::2]]

# For Epochs
l_data = []
l_timestamps = []

# For Events
behavioral_events_array = np.empty(times.shape[0])
behavioral_events_array[:] = np.nan

# For TimeSeries
freezing_times = np.zeros(times.shape[0])


for behavior in l_behaviors:
    behavior_id = l_behaviors.index(behavior) +1
    df_temp = d_dfs['Behaviour'][['{}_1'.format(behavior), '{}_2'.format(behavior)]].copy()
    for i in range(df_temp.shape[0]):

        start_time = df_temp.loc[i, '{}_1'.format(behavior)]
        stop_time = df_temp.loc[i, '{}_2'.format(behavior)]
        if start_time > 0:
            # For events
            l_data.append(behavior_id)
            l_timestamps.append(start_time)
            
            l_data.append(-1)
            l_timestamps.append(stop_time)
            
            
            start_idx = find_nearest(times, start_time)
            stop_idx = find_nearest(times, stop_time)
            
            behavioral_events_array[start_idx:stop_idx] = behavior_id
            freezing_times[start_idx:stop_idx] = behavior_id            
        
        else:
            continue

# Some more stuff for epochs:           
l_sorted_epochs = list(zip(l_data, l_timestamps))
l_sorted_epochs.sort(key=take2nd)

l_data = [elem[0] for elem in l_sorted_epochs]
l_timestamps = [elem[1] for elem in l_sorted_epochs]


# Some more stuff for events:
control = np.copy(behavioral_events_array)
control[np.where(np.isnan(control) == True)] = 0

l_control_description = ['0: no behavior scored']

for behavior_id in np.unique(behavioral_events_array[np.where(behavioral_events_array>0)]):
    behavior = l_behaviors[int(behavior_id) -1]
    l_control_description.append('{}: {}'.format(int(behavior_id), behavior))


control_description = np.asarray(l_control_description)

In [20]:
control_description

array(['0: no behavior scored', '1: Immobility', '2: Rearing',
       '3: StretchAttend', '4: Grooming'], dtype='<U21')

In [21]:
#from pynwb.behavior import BehavioralEpochs

#behaviorl_epochs_obj = BehavioralEpochs()
#behaviorl_epochs_obj.create_interval_series('behavioral epochs', data=np.asarray(l_data), timestamps=np.asarray(l_timestamps))

In [22]:
from pynwb.behavior import BehavioralEvents

events_obj = BehavioralEvents()
events_obj.create_timeseries('Behavioral Events - (1: Freezing, 2: Rearing, 3: StretchAttend, 4: Grooming)', 
                             data=behavioral_events_array, timestamps=times, control=control, control_description = control_description, unit='behavior', continuity='step')

Behavioral Events - (1: Freezing, 2: Rearing, 3: StretchAttend, 4: Grooming) pynwb.base.TimeSeries at 0x140226867859360
Fields:
  comments: no comments
  continuity: step
  control: [0. 0. 0. ... 0. 0. 0.]
  control_description: ['0: no behavior scored' '1: Immobility' '2: Rearing' '3: StretchAttend'
 '4: Grooming']
  conversion: 1.0
  data: [nan nan nan ... nan nan nan]
  description: no description
  interval: 1
  resolution: -1.0
  timestamps: [3.33330000e-02 6.66660000e-02 9.99990000e-02 ... 8.99791002e+02
 8.99824335e+02 8.99857668e+02]
  timestamps_unit: seconds
  unit: behavior

In [23]:
#from pynwb.behavior import BehavioralTimeSeries

#timeseries_obj = BehavioralTimeSeries()
#timeseries_obj.create_timeseries('Immobility times', data=freezing_times, timestamps=times, unit='behavior', continuity='step')

### Extract HeartRate data

In [24]:
timestamps = d_dfs['HeartRate']['Times'].values
data = d_dfs['HeartRate']['HeartRate'].values

heartrate_obj = TimeSeries('Heart rate recording', data=data, timestamps=timestamps, unit='beats per minute')

### Get some dummy thermal data

In [25]:
timestamps = df_states.loc[(df_states['Session'] == 'OF') & (df_states['Animal_ID'] == '175_F4-37'), 'Times'].values
temperature = df_states.loc[(df_states['Session'] == 'OF') & (df_states['Animal_ID'] == '175_F4-37'), 'Temperature'].values

temperature_obj = TimeSeries('Thermal recording', data=temperature, timestamps=timestamps, unit='degrees celsius')

## Setup the NWB file:

In [27]:
tz = pytz.timezone('Europe/Berlin')
N = 12

nwbfile = NWBFile(
    session_description='Open Field recording',
    session_id = 'OF',
    surgery = 'Virus injection on 2020/08/27 by Nina Schukraft (stereotactical coordinates: AP=-4.6, ML=0.55, DV=-3.0 [units: mm])\nGRIN-lense implantation on 2020/09/07 by Nina Schukraft (stereotactical coordinates: AP=-4.6, ML=0.55, DV=-2.5 [units: mm])',
    virus = 'php.eb-flex-GCaMP7f, injection location: AP=-4.6, ML=0.55, DV=-3.0 [units: mm], source: in-house production',
    identifier=''.join(random.choices(string.ascii_uppercase + string.digits, k=N)),
    session_start_time=datetime.datetime(2020,10,30,9,30, tzinfo=tz),
    experimenter = ['Nina Schukraft', 'Jérémy Signoret-Genest', 'Philip Tovote'],
    lab = 'Defense Circuits Lab',
    institution = 'University Hospital Wuerzburg, Institute of Clinical Neurobiology'
)


nwbfile.subject = Subject(
    subject_id = '175_F7-49',
    age = 'P148D', 
    date_of_birth = datetime.datetime(2020, 6, 4, tzinfo=tz),
    strain = 'B6J.129S6(FVB)-Slc17a6tm2(cre)Lowl/MwarJ',
    description = 'Mouse #F7-49 of line 175 (Vglut2-ires-cre)',
    genotype = 'tg/+',
    species = 'Mus musculus', 
    sex = 'M'
)


device = Device(name='Miniscope', description='NVista3.0', manufacturer='Inscopix, US')
nwbfile.add_device(device)
optical_channel = OpticalChannel('my_optchan', 'description', 500.)
imaging_plane = nwbfile.create_imaging_plane('my_imgpln', optical_channel,
                                             description='ventro-lateral periaqueductal gray (AP=-4.6, ML=0.55, DV=-3.0)',
                                             device=device, excitation_lambda=475., imaging_rate=10., indicator='GCamP6s',
                                             location='vlPAG',
                                             unit='millimeter')
                                             #reference_frame='A frame to refer to',
                                             #grid_spacing=(.01, .01))


image_series = TwoPhotonSeries(name='CaI', data=f['mov'][:],
                               dimension=[2],
                               imaging_plane=imaging_plane,
                               starting_frame=[0], format='tiff', starting_time=0.0, rate=1.0, unit='millimeter')

nwbfile.add_acquisition(image_series)


mod = nwbfile.create_processing_module('ophys', 'contains optical physiology processed data')
img_seg = ImageSegmentation()
mod.add(img_seg)
ps = img_seg.create_plane_segmentation('ROI segmentations',
                                       imaging_plane, 'my_planeseg', image_series)

ID = 0
for ROI_ID in range(len(l_ROI_masks)):
    if ROI_ID in [3, 4, 10, 12, 14, 16, 22, 25]:
        continue
    else:
        ps.add_roi(image_mask=l_ROI_masks[ROI_ID][1], id=ID)
        ID = ID+ 1
        
        
l_ROI_IDs_included = []
l_ROI_IDs_excluded = []

for ROI_ID in range(len(l_ROI_masks)):
    if ROI_ID in [3, 4, 10, 12, 14, 16, 22, 25]:
        l_ROI_IDs_excluded.append(l_ROI_masks[ROI_ID][0])
    else:
        l_ROI_IDs_included.append(l_ROI_masks[ROI_ID][0])
        
        
fl = Fluorescence()
mod.add(fl)

rt_region = ps.create_roi_table_region(description='all ROIs')
data_included = d_dfs['CAI - Traces'][l_ROI_IDs_included].values
data_excluded = d_dfs['CAI - Traces'][l_ROI_IDs_excluded].values
timestamps = d_dfs['CAI - Traces']['Times'].values
rrs = fl.create_roi_response_series('included', data=data_included, rois=rt_region, unit='lumens', timestamps=timestamps)
#fl.create_roi_response_series('excluded', data=data_excluded, rois=rt_region, unit='lumens', timestamps=timestamps)

In [28]:
nwb2widget(nwbfile)

## Add all calcium imaging data

## Add all behavioral data

#### Attempt to include raw RGB movie as acquisition file

import cv2
cap = cv2.VideoCapture('175_F7-49_201030_OF.AVI')

l_frames = []
for i in range(500):
    cap.set(cv2.CAP_PROP_POS_FRAMES,i)
    a,b=cap.read()
    l_frames.append(cv2.cvtColor(b, cv2.COLOR_BGR2GRAY))
    
    
raw_behavior_movie = np.asarray(l_frames)

raw_behavior_movie.shape

import h5py

with h5py.File('resize_dataset.hdf5', 'w') as f:
    f.create_dataset('raw RGB movie',  data=raw_behavior_movie, maxshape=(None, 480, 640))

with h5py.File('resize_dataset.hdf5', 'a') as f:
    print(f['raw RGB movie'][:].shape)


with h5py.File('resize_dataset.hdf5', 'a') as f:
    f['raw RGB movie'].resize(600, axis=0)
    f['raw RGB movie'][500:600] = raw_behavior_movie[:100]
    print(f['raw RGB movie'][:].shape)

with h5py.File('resize_dataset.hdf5', 'a') as f:
    print(f['raw RGB movie'][:].shape)

from pynwb.image import ImageSeries

behavior_movie_obj = ImageSeries('top view - grayscale2', format='external', external_file=['resize_dataset.hdf5'], rate = 900/27000, starting_time=0.0)


nwbfile.add_acquisition(behavior_movie_obj)

from pynwb.image import ImageSeries

behavior_movie_obj = ImageSeries('top view - grayscale2', data=raw_behavior_movie, rate = 900/27000, starting_time=0.0)

nwbfile.add_acquisition(behavior_movie_obj)

from pynwb.image import OpticalSeries

behavior_mov_obj = OpticalSeries('top view - grayscale3', data=raw_behavior_movie, rate=900/27000, starting_time=0.0, 
                                 distance=0.3, field_of_view=[1, 1], orientation='up is up')

nwbfile.add_acquisition(behavior_mov_obj)

### Tracking:

In [29]:
# Create a SpatialSeries that contains the data - extension of TimeSeries
spatial_series_obj = SpatialSeries(
    name = 'SpatialSeries', 
    description = '(x,y) position in open field',
    data = position_data,
    timestamps = position_times,   # if no timestamps are provided, session_start_time from file setup will be used - ?
    reference_frame = '(0,0) is bottom left corner'
)

# Create a container "Position" that can contain multiple 
# SpatialSeries - e.g. if multiple trials are used? not sure though
position_obj = Position(spatial_series=spatial_series_obj) # name is set to 'Position' by default

# Create a "Processing_module" to store the behavioral data, 
# since it is not considered as raw due to preprocessing 
# by other alglorithms / softwares
behavior_module = nwbfile.create_processing_module(
    name='behavior', 
    description='processed behavioral data'
)

# Add the Processing_module to the NWB:N file
behavior_module.add(position_obj)

#behavior_module.add(behaviorl_epochs_obj)



#behavior_module.add(timeseries_obj)

Position pynwb.behavior.Position at 0x140225167849984
Fields:
  spatial_series: {
    SpatialSeries <class 'pynwb.behavior.SpatialSeries'>
  }

### Events:

In [30]:
behavior_module.add(events_obj)

BehavioralEvents pynwb.behavior.BehavioralEvents at 0x140226460019728
Fields:
  time_series: {
    Behavioral Events - (1: Freezing, 2: Rearing, 3: StretchAttend, 4: Grooming) <class 'pynwb.base.TimeSeries'>
  }

## Add HeartRate recordings

In [31]:
hr_mod = nwbfile.create_processing_module('cardiac', 'processed heart rate recording data')
hr_mod.add(heartrate_obj)

Heart rate recording pynwb.base.TimeSeries at 0x140226453605200
Fields:
  comments: no comments
  conversion: 1.0
  data: [         nan          nan          nan ... 680.01511145 678.98906073
 676.43742954]
  description: no description
  interval: 1
  resolution: -1.0
  timestamps: [9.300000e-02 1.738000e-01 2.536000e-01 ... 8.996550e+02 8.997442e+02
 8.998342e+02]
  timestamps_unit: seconds
  unit: beats per minute

## Add thermal data (dummy data)

In [32]:
temp_mod = nwbfile.create_processing_module('thermal', 'processed temperature recording data')
temp_mod.add(temperature_obj)

Thermal recording pynwb.base.TimeSeries at 0x140226450680512
Fields:
  comments: no comments
  conversion: 1.0
  data: [26.46465454 26.45976227 26.45495887 ... 34.07843137 34.07843137
 34.07843137]
  description: no description
  interval: 1
  resolution: -1.0
  timestamps: [  5.75   6.     6.25 ... 899.25 899.5  899.75]
  timestamps_unit: seconds
  unit: degrees celsius

## Inspect the generated NWB file using NWB widgets

In [33]:
nwb2widget(nwbfile)

In [34]:
with NWBHDF5IO(file_dir + 'DCL_CAI_data.nwb', 'w') as io:
    io.write(nwbfile)

/home/ds/anaconda3/envs/NWB2/lib/python3.9/site-packages/hdmf/build/objectmapper.py:256: DtypeConversionWarning:

Spec 'TimeSeries/control': Value with data type float64 is being converted to data type uint64 (min specification: uint8).



In [35]:
with NWBHDF5IO(file_dir + 'DCL_CAI_data.nwb', 'r') as io:
    read_nwbfile = io.read()

In [36]:
nwb2widget(read_nwbfile)

In [24]:
nwb2widget(nwbfile)